### Import the necessary variables

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import math
import seaborn as sns
import impyute as impy

### Jupyter configuration

In [2]:
pd.set_option('display.max_rows', 500)
%matplotlib inline

### Load the dataset for all the countries

In [3]:
def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

In [4]:
all_countries_dataset = pd.read_csv('all_countries_dataset.csv')

### List all available variables

In [5]:
all_countries_dataset.columns.tolist()

['Unnamed: 0',
 'iso_code',
 'continent',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'tests_units',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_

### Cap 3:

### Fix variables datatypes

In [6]:
# drop cloumn Unnamed: 0

if 'Unnamed: 0' in all_countries_dataset.columns:
    all_countries_dataset = all_countries_dataset.drop(columns=['Unnamed: 0'])

# Fixing date column datatype
all_countries_dataset['date'] = pd.to_datetime(all_countries_dataset['date']);
# all_countries_dataset['location'] = all_countries_dataset['location'].astype('category');
# all_countries_dataset['continent'] = all_countries_dataset['continent'].astype('category');

# all_countries_dataset.set_index('date', inplace=True)

### Exclude unnecessary continents

In [7]:
rule = np.logical_or(all_countries_dataset['continent'] == 'Europe', all_countries_dataset['location'] == 'United States')
dataset = all_countries_dataset[rule]

### Select only the relevant variables that can be used

In report will be needed to be explained why we excluded those variables

In [8]:
variables = [
    'continent',
    'location',
    'date',
    'new_cases',
    'new_deaths',
    'icu_patients',
    'new_tests',
    'positive_rate',
    'people_vaccinated',
    'new_vaccinations',
    'total_boosters',
    'stringency_index',
    'population',
    'population_density',
    'cardiovasc_death_rate',
    'diabetes_prevalence',
    'human_development_index'
 ]

Exclude the unnecessary variables

In [9]:
dataset = dataset[variables]

### Detect the microcountrie (countries that have a population of less than 500 000)

In [10]:
countries = dataset['location'].unique()

population_threshold = 500000
micro_countries = []

for country in countries:
    country_population = dataset[dataset['location'] == country]['population'].max()
    if country_population < population_threshold:
        micro_countries.append(country)

print(len(micro_countries))
micro_countries

11


['Andorra',
 'Faeroe Islands',
 'Gibraltar',
 'Guernsey',
 'Iceland',
 'Isle of Man',
 'Jersey',
 'Liechtenstein',
 'Monaco',
 'San Marino',
 'Vatican']

### Exclude the microcountries

In [11]:
# exclude the micro countries
dataset = dataset[~dataset['location'].isin(micro_countries)]

### Feature scaling

In [12]:
# Perform feature scaling

# take only numerical variables
# numerical_variables = [
#     'new_cases',
#     'new_deaths',
#     'reproduction_rate',
#     'icu_patients',
#     'hosp_patients',
#     'new_tests',
#     'positive_rate',
#     'people_vaccinated',
#     'new_vaccinations',
#     'total_boosters',
#     'stringency_index',
#     'population',
#     'population_density',
#     'cardiovasc_death_rate',
#     'diabetes_prevalence',
#     'human_development_index'
# ]

# # take non-numerical variables
# non_numerical_variables = [
#     'continent',
#     'location',
#     'date'
# ]

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()

# obj_dataset = dataset[non_numerical_variables].copy()
# num_dataset = dataset[numerical_variables].copy()
# num_dataset_scaled = sc.fit_transform(num_dataset)
# dataset = pd.concat([obj_dataset, pd.DataFrame(data=num_dataset_scaled, columns=numerical_variables)], axis=1)

# dataset.dtypes


### Replace negative values

In [13]:
# removed records where new_cases is negative
# for each numerical variable 
# if new_cases is negative, set it to 0
def remove_negative_values(dataset):
    new_dataset = dataset.copy()
    for variable in variables:
        if variable in ['location', 'continent', 'date']:
            continue
        for index, row in new_dataset.iterrows():
            if index == 0:
                continue
            if row[variable] < 0 and index in new_dataset.index:
                new_dataset.at[index, variable] = new_dataset.at[index - 1, variable]
    return new_dataset

In [14]:
cleaned_dataset = remove_negative_values(dataset)

### Replace first non missing values

In [15]:
def replace_backwards(new_dataset, index, variable, value_to_replace_with):
    while index in new_dataset.index:
        new_dataset.at[index, variable] = value_to_replace_with
        index -= 1

# replace first missing values with first non missing value
def replace_first_missing_values(dataset):
    new_dataset = dataset.copy()
    
    for variable in ['population', 'population_density', 'cardiovasc_death_rate', 'diabetes_prevalence', 'human_development_index']:
        for index, row in dataset.iterrows():
            if not pd.isna(row[variable]):
                replace_backwards(new_dataset, index, variable, row[variable])
                break
    
    for variable in ['new_vaccinations', 'people_vaccinated', 'total_boosters', 'icu_patients', 'new_tests', 'new_cases', 'new_deaths', 'positive_rate', 'stringency_index']:
        for index, row in dataset.iterrows():
            if not pd.isna(row[variable]):
                replace_backwards(new_dataset, index, variable, 0)
                break

    return new_dataset

In [16]:
cleaned_dataset = replace_first_missing_values(cleaned_dataset)

### Replace missing values


In [17]:
def next_non_missing_value(dataset, index, variable):
    next_index = index
    while next_index in dataset.index and pd.isna(dataset.loc[next_index, variable]):
        next_index += 1
    if next_index not in dataset.index:
        return (next_index - 1, -1)
    return (index, dataset.loc[next_index, variable])

def last_non_missing_value(dataset, index, variable):
    last_index = index
    while last_index in dataset.index and pd.isna(dataset.loc[last_index, variable]):
        last_index -= 1
    if last_index not in dataset.index:
        return (last_index + 1, -1)
    return (index, dataset.loc[last_index, variable])

def replace_missing_values(dataset):
    df = dataset.copy()
    new_dataset = pd.DataFrame()
    for country in df['location'].unique():
        country_dataset = df[df['location'] == country]
        for variable in variables:
            if variable in ['location', 'continent', 'date']:
                continue
            found_non_missing = False
            for index, row in country_dataset.iterrows():
                if pd.isna(row[variable]):
                    if found_non_missing and (index - 1) in country_dataset.index and (index + 1) in country_dataset.index:
                        next = next_non_missing_value(country_dataset, index, variable)
                        last = last_non_missing_value(country_dataset, index, variable)
                        if next[1] == -1 or last[1] == -1:
                            continue
                        new_val = (next[1] - last[1]) / 2
                        country_dataset.loc[index, variable] = last[1] + new_val
                else:
                    found_non_missing = True
        new_dataset = pd.concat([new_dataset, country_dataset])
    return new_dataset

In [18]:
cleaned_dataset = replace_missing_values(cleaned_dataset)

/home/ion/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### Remove anomalies

In [19]:
def split_dataframe(a, n): 
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [20]:
def get_mean(dataset, variable):
    sum = 0
    max = dataset[variable].max()
    if pd.isna(max):
        return 0
    count = 0
    for index, row in dataset.iterrows():
        if (~pd.isna(row[variable]) and row[variable] < max) == True:
            sum = sum + row[variable]
            count = count + 1
    if sum == 0:
        return 0
    return sum / count

In [21]:
def remove_anomalies(dataset):
    new_dataset = pd.DataFrame()
    for country in dataset['location'].unique():
        new_country_data = pd.DataFrame()
        country_data = dataset[dataset['location'] == country]
        dataset_chunks = split_dataframe(country_data, 25)

        for chunk in dataset_chunks:
            for variable in variables:
                if variable in ['location', 'continent', 'date']:
                    continue
                anomaly_indexes = chunk[chunk[variable] > chunk[variable].quantile(0.99)].index
                for index in anomaly_indexes:
                    if index - 1 in chunk.index:
                        chunk.at[index, variable] = chunk.at[index - 1, variable]
            new_country_data = new_country_data.append(chunk)
        
        new_dataset = new_dataset.append(new_country_data)

    return new_dataset

In [22]:
cleaned_dataset = remove_anomalies(cleaned_dataset)

### Show the coverage percentage for hosp_patiens and icu_patients

In [23]:
def show_coverage(dataset, variables):
    for country in dataset['location'].unique():
        for variable in variables:
            count = 0
            country_data = dataset[dataset['location'] == country]
            for index, row in country_data.iterrows():
                if not pd.isna(row[variable]):
                    count = count + 1
            covered = count / country_data.shape[0] * 100
            print("{}, {} = {} %".format(country, variable, covered))
        print("\n")

In [24]:
show_coverage(cleaned_dataset, ['icu_patients'])

Albania, icu_patients = 0.0 %


Austria, icu_patients = 98.7012987012987 %


Belarus, icu_patients = 0.0 %


Belgium, icu_patients = 92.4646781789639 %


Bosnia and Herzegovina, icu_patients = 0.0 %


Bulgaria, icu_patients = 93.87417218543047 %


Croatia, icu_patients = 0.0 %


Cyprus, icu_patients = 98.50993377483444 %


Czechia, icu_patients = 97.57281553398059 %


Denmark, icu_patients = 89.2018779342723 %


Estonia, icu_patients = 95.141065830721 %


Finland, icu_patients = 90.6687402799378 %


France, icu_patients = 98.30246913580247 %


Germany, icu_patients = 90.54263565891473 %


Greece, icu_patients = 0.0 %


Hungary, icu_patients = 0.0 %


Ireland, icu_patients = 94.28104575163398 %


Italy, icu_patients = 95.00780031201248 %


Kosovo, icu_patients = 0.0 %


Latvia, icu_patients = 0.0 %


Lithuania, icu_patients = 0.0 %


Luxembourg, icu_patients = 98.70340356564019 %


Malta, icu_patients = 98.35255354200989 %


Moldova, icu_patients = 0.0 %


Montenegro, icu_patients = 0.0

Remove countries that don't have data for icu_patients variable

In [25]:
def remove_countries_without_data(dataset, variables):
    new_dataset = pd.DataFrame()

    for country in dataset['location'].unique():
        for variable in variables:
            count = 0
            country_data = dataset[dataset['location'] == country]
            for index, row in country_data.iterrows():
                if not pd.isna(row[variable]):
                    count = count + 1
            covered = count / country_data.shape[0] * 100
            if covered != 0:
                new_dataset = new_dataset.append(country_data)
    return new_dataset

In [26]:
cleaned_dataset = remove_countries_without_data(cleaned_dataset, ['icu_patients'])

In [27]:
cleaned_dataset['location'].unique()

array(['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Denmark',
       'Estonia', 'Finland', 'France', 'Germany', 'Ireland', 'Italy',
       'Luxembourg', 'Malta', 'Netherlands', 'Portugal', 'Romania',
       'Serbia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'United Kingdom', 'United States'], dtype=object)

In [28]:
cleaned_dataset.to_csv('cleaned_dataset.csv')

In [29]:
cleaned_dataset.corr()

,new_cases,new_deaths,icu_patients,new_tests,positive_rate,people_vaccinated,new_vaccinations,total_boosters,stringency_index,population,population_density,cardiovasc_death_rate,diabetes_prevalence,human_development_index
new_cases,1.000000,0.820677,0.900230,0.815766,0.129381,0.499717,0.490728,0.441331,0.138078,0.681693,-0.090770,-0.077306,0.244940,0.083081
new_deaths,0.820677,1.000000,0.872676,0.731426,0.162336,0.402510,0.540087,0.512688,0.232848,0.724571,-0.099187,-0.074018,0.258373,0.071535
icu_patients,0.900230,0.872676,1.000000,0.806052,0.122777,0.600376,0.532213,0.488099,0.184524,0.848064,-0.109645,-0.087014,0.295700,0.084737
new_tests,0.815766,0.731426,0.806052,1.000000,-0.039072,0.690078,0.705803,0.556710,0.145869,0.784560,-0.086071,-0.165494,0.195498,0.161550
positive_rate,0.129381,0.162336,0.122777,-0.039072,1.000000,-0.067981,-0.058312,0.089620,0.240261,0.019895,-0.088177,0.283651,0.108422,-0.217305
people_vaccinated,0.499717,0.402510,0.600376,0.690078,-0.067981,1.000000,0.590791,0.673678,-0.011384,0.845560,-0.105187,-0.133422,0.285961,0.139127
new_vaccinations,0.490728,0.540087,0.532213,0.705803,-0.058312,0.590791,1.000000,0.319658,0.130647,0.753459,-0.106249,-0.154487,0.255182,0.142347
total_boosters,0.441331,0.512688,0.488099,0.556710,0.089620,0.673678,0.319658,1.000000,-0.048601,0.618741,-0.092905,-0.000012,0.282262,0.040666
stringency_index,0.138078,0.232848,0.184524,0.145869,0.240261,-0.011384,0.130647,-0.048601,1.000000,0.109724,0.019635,-0.138047,0.061515,0.032648
population,0.681693,0.724571,0.848064,0.784560,0.019895,0.845560,0.753459,0.618741,0.109724,1.000000,-0.133020,-0.137692,0.352960,0.143607


### Plots

In [30]:
def plot_vars_per_country(dataset, cleaned_dataset):
    for continent in dataset['continent'].unique():
        print(continent)

        continent_data = dataset[dataset['continent'] == continent]
        continent_data_cleaned = cleaned_dataset[cleaned_dataset['continent'] == continent]

        for variable in variables:
            if variable == 'location' or variable == 'continent' or variable == 'date':
                continue
            continent_countries = continent_data['location'].unique()

            # set fig size
            plt.figure(figsize=(50, 20))

            for country in continent_countries:
                country_data = continent_data[continent_data['location'] == country]
                plt.plot(country_data['date'], country_data[variable], label=country)

            plt.legend()
            plt.title("{} - {}".format(continent, variable))
            plt.show()

            # set fig size
            plt.figure(figsize=(50, 20))

            for country in continent_countries:
                country_data_cleaned = continent_data_cleaned[continent_data_cleaned['location'] == country]
                plt.plot(country_data_cleaned['date'], country_data_cleaned[variable], label=country)

            plt.legend()
            plt.title("{} - {}".format(continent, variable))
            plt.show()


In [31]:
# plot_vars_per_country(dataset, cleaned_dataset)

VARMAX

In [32]:
# dataset - diff_dataset
def inverse_differenciate(dataset):
    # for variable in dataset.columns:
    #     if variable == 'location' or variable == 'continent' or variable == 'date':
    #         continue
    #     dataset[variable] = cleaned_dataset[variable].shift(1) + dataset[variable]

    for variable in dataset.columns:
        if variable == 'location' or variable == 'continent' or variable == 'date':
            continue
        dataset[variable] = pd.Series(np.r_[cleaned_dataset[variable], dataset[variable]].cumsum())

    return dataset

In [33]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [34]:
# take only numerical variables
numerical_variables = [
    'new_cases',
    'new_deaths',
    'icu_patients',
    'new_tests',
    'positive_rate',
    'people_vaccinated',
    'new_vaccinations',
    'total_boosters',
    'stringency_index',
    'population',
    'population_density',
    'cardiovasc_death_rate',
    'diabetes_prevalence',
    'human_development_index'
]

# take non-numerical variables
non_numerical_variables = [
    'continent',
    'location',
    'date'
]

varmax_dataset = cleaned_dataset.copy()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

obj_dataset = varmax_dataset[non_numerical_variables].copy()
num_dataset = varmax_dataset[numerical_variables].copy()
# num_dataset = sc.fit_transform(num_dataset)

num_dataset = pd.DataFrame(num_dataset, columns=numerical_variables)


num_dataset = np.log(num_dataset)

# Take First Difference to Remove Trend
num_dataset = num_dataset.diff()

num_dataset = num_dataset.diff()

# Remove Increasing Volatility
# num_dataset = num_dataset.groupby(num_dataset.index.year).std()

varmax_dataset = pd.concat([obj_dataset, pd.DataFrame(data=num_dataset, columns=numerical_variables)], axis=1)

varmax_dataset.dtypes

continent                          object
location                           object
date                       datetime64[ns]
new_cases                         float64
new_deaths                        float64
icu_patients                      float64
new_tests                         float64
positive_rate                     float64
people_vaccinated                 float64
new_vaccinations                  float64
total_boosters                    float64
stringency_index                  float64
population                        float64
population_density                float64
cardiovasc_death_rate             float64
diabetes_prevalence               float64
human_development_index           float64
dtype: object

### Split into traing and test sets

In [35]:
varmax_dataset.index = pd.to_datetime(varmax_dataset.date)
varmax_dataset.index.freq = varmax_dataset.index.inferred_freq

In [36]:
def replace_nans(dataset):
    # for each country
    new_dataset = pd.DataFrame()

    for country in dataset['location'].unique():
        country_dataset = dataset[dataset['location'] == country]

        for variable in numerical_variables:
            for index, row in country_dataset.iterrows():
                if pd.isna(row[variable]) or row[variable] == float('inf') or row[variable] == float('-inf'):
                    previous_timestamp = index - pd.Timedelta(days=1)
                    if previous_timestamp in country_dataset.index and pd.isna(country_dataset.loc[previous_timestamp, variable]) == False:
                        country_dataset.at[index, variable] = country_dataset.at[previous_timestamp, variable]
                    else:
                        country_dataset.at[index, variable] = 0
                        
        new_dataset = pd.concat([new_dataset, country_dataset], axis=0)

    return new_dataset

In [37]:
varmax_dataset = replace_nans(varmax_dataset)

In [38]:
# split into train and test
training_date_limit = date(2021, 8, 1)

varmax_dataset.index = pd.to_datetime(varmax_dataset.date)
varmax_dataset.index.freq = varmax_dataset.index.inferred_freq

varmax_train_dataset = varmax_dataset[varmax_dataset['date'].dt.date < training_date_limit]
varmax_test_dataset = varmax_dataset[varmax_dataset['date'].dt.date >= training_date_limit]

In [39]:
import statsmodels.api as sm

exogeneous_variables = [
    'population',
    'population_density',
    'diabetes_prevalence',
    'human_development_index',
    'new_tests',
    'stringency_index',
    'icu_patients',
    'cardiovasc_death_rate',
    'people_vaccinated',
    'new_vaccinations',
    'total_boosters'
]

endogeneous_variables = [
    'new_cases',
    'new_deaths',
    'positive_rate'
]

In [40]:
print(varmax_train_dataset.location.unique())

['Austria' 'Belgium' 'Bulgaria' 'Cyprus' 'Czechia' 'Denmark' 'Estonia'
 'Finland' 'France' 'Germany' 'Ireland' 'Italy' 'Luxembourg' 'Malta'
 'Netherlands' 'Portugal' 'Romania' 'Serbia' 'Slovenia' 'Spain' 'Sweden'
 'Switzerland' 'United Kingdom' 'United States']


### ACF and PACF

https://www.youtube.com/watch?v=CAT0Y66nPhs&ab_channel=DataScienceShow

In [41]:
print(len(numerical_variables))

14


In [42]:
import matplotlib.pyplot as plt

def plot_acf_per_country(dataset):
    for country in dataset['location'].unique():
        country_dataset = dataset[dataset['location'] == country]

        print('\n')
        fig = plt.figure(figsize=(30, 15))
        for index, variable in enumerate(numerical_variables):
            # add plot_acf to subplot for each variable
            ax = fig.add_subplot(4, 4, index + 1)
            plot_acf(country_dataset[variable], ax=ax, lags=300)
            ax.set_title(variable)

        fig.suptitle(country)
        fig.show()

def plot_pacf_per_country(dataset):
    for country in dataset['location'].unique():
        country_dataset = dataset[dataset['location'] == country]

        print('\n')
        fig = plt.figure(figsize=(30, 15))
        for index, variable in enumerate(numerical_variables):
            # add plot_acf to subplot for each variable
            ax = fig.add_subplot(4, 4, index + 1)
            plot_pacf(country_dataset[variable], ax=ax)
            ax.set_title(variable)

        fig.suptitle(country)
        fig.show()

In [43]:
# plot_acf_per_country(varmax_train_dataset)

In [44]:
us_dataset = varmax_train_dataset[varmax_train_dataset['location'] == 'United States']
us_dataset.corr()

,new_cases,new_deaths,icu_patients,new_tests,positive_rate,people_vaccinated,new_vaccinations,total_boosters,stringency_index,population,population_density,cardiovasc_death_rate,diabetes_prevalence,human_development_index
new_cases,1.000000,0.532344,0.227044,0.484105,0.521808,0.029325,0.458681,NaN,0.190351,0.0,0.0,0.0,0.0,0.0
new_deaths,0.532344,1.000000,0.182087,0.521559,0.221784,0.062535,0.319239,NaN,-0.104517,0.0,0.0,0.0,0.0,0.0
icu_patients,0.227044,0.182087,1.000000,0.214937,0.124783,-0.003075,0.140769,NaN,0.017485,0.0,0.0,0.0,0.0,0.0
new_tests,0.484105,0.521559,0.214937,1.000000,0.124143,0.072706,0.594651,NaN,0.123621,0.0,0.0,0.0,0.0,0.0
positive_rate,0.521808,0.221784,0.124783,0.124143,1.000000,-0.021063,0.136490,NaN,0.075846,0.0,0.0,0.0,0.0,0.0
people_vaccinated,0.029325,0.062535,-0.003075,0.072706,-0.021063,1.000000,0.146876,NaN,0.000166,0.0,0.0,0.0,0.0,0.0
new_vaccinations,0.458681,0.319239,0.140769,0.594651,0.136490,0.146876,1.000000,NaN,0.053066,0.0,0.0,0.0,0.0,0.0
total_boosters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stringency_index,0.190351,-0.104517,0.017485,0.123621,0.075846,0.000166,0.053066,NaN,1.000000,0.0,0.0,0.0,0.0,0.0
population,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,1.0,-1.0,1.0,1.0,-1.0


### Dickey-fuller test

In [45]:
# augmented dickey-fuller test
from statsmodels.tsa.stattools import adfuller

def dickey_fuller_test(dataset):
    for country in dataset.location.unique():
        country_dataset = varmax_train_dataset[varmax_train_dataset['location'] == country]
        print("{}".format(country))
        for variable in numerical_variables:
            print("{}".format(variable))

            X = np.asarray(country_dataset[variable])
            result = adfuller(X)

            print('ADF Statistic: %f' % result[0])
            print('p-value: %f' % result[1])
            print('Critical Values:')
            for key, value in result[4].items():
                print('\t%s: %.3f' % (key, value))

            print('\n')

        print('\n')
        print("=============================")


In [46]:
# dickey_fuller_test(varmax_train_dataset)

In [47]:
mod = sm.tsa.VARMAX(np.asarray(varmax_train_dataset[endogeneous_variables]), np.asarray(varmax_train_dataset[exogeneous_variables]), order=(2, 0))

In [48]:
res = mod.fit(disp=True)
res.summary()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           60     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.22793D+00    |proj g|=  3.48879D-01


 This problem is unconstrained.



At iterate    1    f=  2.22705D+00    |proj g|=  2.20836D-01

At iterate    2    f=  2.22665D+00    |proj g|=  4.50807D-01

At iterate    3    f=  2.22608D+00    |proj g|=  3.20898D-02

At iterate    4    f=  2.22606D+00    |proj g|=  2.27328D-02

At iterate    5    f=  2.22601D+00    |proj g|=  4.66597D-02

At iterate    6    f=  2.22589D+00    |proj g|=  1.06360D-01

At iterate    7    f=  2.22567D+00    |proj g|=  1.55558D-01

At iterate    8    f=  2.22528D+00    |proj g|=  1.98106D-01

At iterate    9    f=  2.22515D+00    |proj g|=  1.43627D-01

At iterate   10    f=  2.22491D+00    |proj g|=  5.27385D-02

At iterate   11    f=  2.22484D+00    |proj g|=  2.51899D-02

At iterate   12    f=  2.22481D+00    |proj g|=  4.02150D-02

At iterate   13    f=  2.22479D+00    |proj g|=  4.96185D-02

At iterate   14    f=  2.22467D+00    |proj g|=  8.16515D-02

At iterate   15    f=  2.22458D+00    |proj g|=  8.22634D-02

At iterate   16    f=  2.22448D+00    |proj g|=  4.18123D-02

At iter

In [ ]:
# set index to be date
varmax_train_dataset.index = pd.to_datetime(varmax_train_dataset.date)
varmax_train_dataset.index.freq = varmax_train_dataset.index.inferred_freq

varmax_train_dataset

,continent,location,date,new_cases,new_deaths,icu_patients,new_tests,positive_rate,people_vaccinated,new_vaccinations,total_boosters,stringency_index,population,population_density,cardiovasc_death_rate,diabetes_prevalence,human_development_index
date,,,,,,,,,,,,,,,,,
2020-02-25,Europe,Austria,2020-02-25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-26,Europe,Austria,2020-02-26,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-27,Europe,Austria,2020-02-27,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,Europe,Austria,2020-02-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-29,Europe,Austria,2020-02-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-27,North America,United States,2021-07-27,-1.470977,-0.609786,-0.007285,-0.190991,-0.030678,0.000167,-0.821303,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-07-28,North America,United States,2021-07-28,0.249292,-0.691517,-0.018479,-0.211400,-0.027202,0.000044,-0.060950,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-07-29,North America,United States,2021-07-29,0.130673,0.011384,0.013113,-0.062974,-0.000169,0.000027,-0.025316,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
real_us_data_endog = np.asarray(varmax_train_dataset[varmax_train_dataset['location'] == 'United States'][endogeneous_variables])
real_us_data_exog = np.asarray(varmax_train_dataset[varmax_train_dataset['location'] == 'United States'][exogeneous_variables])

In [ ]:
# for each real_us_data_exog, forecast

forecasted = []
# for each item from real_us_data_exog
for index, item in enumerate(real_us_data_exog):
    # forecast
    forecast = res.forecast(exog=item, steps=1)
    # append forecast to forecasted
    forecasted.append(forecast[0])

print(np.asarray(forecasted))


[[-0.84932661 -0.04622345  0.0696697 ]
 [-0.30272051 -0.03548858  0.04723351]
 [-0.57602356 -0.04085602  0.0584516 ]
 ...
 [-0.56942528 -0.04599105  0.05859412]
 [-0.56781008 -0.03712918  0.05865513]
 [-0.56497348 -0.08119482  0.0585042 ]]


In [ ]:
# plot forecasted vs real_us_data_endog

In [ ]:
for i in range(len(forecasted)):
    print(real_us_data_endog[i], forecasted[i])

[0. 0. 0.] [-0.84932661 -0.04622345  0.0696697 ]
[0. 0. 0.] [-0.30272051 -0.03548858  0.04723351]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602356 -0.04085602  0.0584516 ]
[0. 0. 0.] [-0.57602

In [ ]:
# calculate the goodness of fit
from sklearn.metrics import mean_squared_error


def calculate_rmse(real_data, forecasted_data):
    return np.sqrt(mean_squared_error(real_data, forecasted_data))

calculate_rmse(real_us_data_endog, forecasted)

0.5805410255700257

In [ ]:
# calculate r2 
from sklearn.metrics import r2_score


def calculate_r2(real_data, forecasted_data):
    return r2_score(real_data, forecasted_data)

calculate_r2(real_us_data_endog, forecasted)

-1.0062678666701002

In [ ]:
print(len(varmax_train_dataset[varmax_train_dataset['location'] == 'United States'][endogeneous_variables].values))

557


In [ ]:
# prediction = res.get_prediction(start=pd.to_datetime(varmax_train_dataset['date'].max()), dynamic=False)
# prediction_mean = prediction.predicted_mean
# prediction_mean = pd.DataFrame(prediction_mean, columns=endogeneous_variables)

### AutoARIMA

In [ ]:
# get the data only for United States
us_dataset = varmax_train_dataset[varmax_train_dataset['location'] == 'United States']

# import autoarima 
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest

# adf_test = ADFTest(alpha=0.05)
# adf_test = adf_test.should_diff(us_dataset['new_deaths'])

constant_variables = [
    'population',
    'population_density',
    'cardiovasc_death_rate',
    'diabetes_prevalence',
    'human_development_index'
]

# pq = {}

# all_country_p_sum = 0
# all_country_q_sum = 0
# for country in varmax_train_dataset['location'].unique():
#     country_train_dataset = varmax_train_dataset[varmax_train_dataset['location'] == country]
#     print("Using auto_arima for {}".format(country))

#     p_sum = 0
#     q_sum = 0

#     country_p_q = {}

#     for variable in numerical_variables:
#         if variable in constant_variables:
#             continue
#         print("Using auto_arima for {}".format(variable))
#         arima_model = auto_arima(country_train_dataset[variable], start_p=0, start_q=0,
#                                     test='adf',
#                                     max_p=5,
#                                     max_q=5,
#                                     m=4,
#                                     trace=True,
#                                     error_action='warn')
#         arima_model.summary()

#         country_p_q[variable] = {
#             'p': arima_model.order[0],
#             'q': arima_model.order[1]
#         }

#         # get p and q from the summary
#         p_sum += arima_model.order[0]
#         q_sum += arima_model.order[1]
    
#         print('\n')

#     pq[country] = country_p_q
    

#     print('\n')

#     p_mean = p_sum / len(numerical_variables)
#     q_mean = q_sum / len(numerical_variables)
#     all_country_p_sum = all_country_p_sum + p_mean
#     all_country_q_sum = all_country_q_sum + q_mean

#     print("p_mean: {}".format(p_mean))
#     print("q_mean: {}".format(q_mean))
#     print('\n')
#     print("=============================")

# print("all_country_p_mean: {}".format(all_country_p_sum / len(varmax_train_dataset['location'].unique())))
# print("all_country_q_mean: {}".format(all_country_q_sum/ len(varmax_train_dataset['location'].unique())))

# print(pq)

In [ ]:
# arima_model.scoring(us_dataset_test['new_deaths'])

In [ ]:
# plot_vars_per_country(dataset, varmax_dataset)